In [1]:
import math
import random

import matplotlib.animation as animation
import matplotlib.pyplot as plt

In [2]:
class Body(object):
    def __init__(self, mass, x, y):
        self.mass = mass
        self.x = x
        self.y = y
        self.a = None
        self.v = None
        
    '''
    Psuedocode from: http://arborjs.org/docs/barnes-hut
    
    1. If the current node is an external node (and it is not body b), calculate the 
       force exerted by the current node on b, and add this amount to b’s net force.
    
    2. Otherwise, calculate the ratio s/d. If s/d < θ, treat this internal node as a 
       single body, and calculate the force it exerts on body b, and add this amount to 
       b’s net force.
    
    3. Otherwise, run the procedure recursively on each of the current node’s children.
    '''
    def compute_acceleration(self, tree, g, theta):
        self.a = [0,0]
        dx = tree.x_cm - self.x
        dy = tree.y_cm - self.y
        r = math.sqrt(dx*dx + dy*dy)
        if r != 0:
            if tree.size < r * theta:
                if tree.b:
                    mag = g * tree.b.mass
                    self.a[0] += mag * dx / (r*r)
                    self.a[1] += mag * dy / (r*r)
                else:
                    for ch in tree.children:
                        self.compute_acceleration(ch, g, theta)
                        
    def compute_velocity(self, time_step=1):
        if self.a:
            self.v = [0,0]
            self.v[0] += self.a[0] * time_step
            self.v[1] += self.a[1] * time_step

In [3]:
'''
1. Build the Quadtree, using QuadtreeBuild 
   as described above.
   
2. For each subsquare in the quadtree, 
   compute the center of mass and total mass 
   for all the particles it contains.
   
3. For each particle, traverse the tree 
   to compute the force on it.
'''
class QuadTree(object):
    def __init__(self, x_min, x_max, y_min, y_max):
        self.b = None
        self.num_bodies = 0
        self.total_mass = 0
        self.x_min = x_min
        self.x_max = x_max
        self.y_min = y_min
        self.y_max = y_max
        self.x_mid = (self.x_min + self.x_max) / 2
        self.y_mid = (self.y_min + self.y_max) / 2
        self.size = x_max - x_min
        self.x_cm = 0
        self.y_cm = 0
        self.children = []
        
    def get_quadrant_index(self, b):
        if b.y > self.y_mid:
            if b.x > self.x_mid:
                return 0 # Q1
            else: 
                return 1 # Q2
        else:
            if b.x < self.x_mid:
                return 2 # Q3
            else:
                return 3 # Q4
    
    def subdivide(self):
        self.children = [QuadTree(self.x_mid, self.x_max, self.y_mid, self.y_max),
                         QuadTree(self.x_min, self.x_mid, self.y_mid, self.y_max),
                         QuadTree(self.x_min, self.x_mid, self.y_min, self.y_mid),
                         QuadTree(self.x_mid, self.x_max, self.y_min, self.y_mid)]
    
    '''
    Psuedocode from: http://arborjs.org/docs/barnes-hut

    1. If node n does not contain a body, put the new body b here.

    2. If node n is an internal node, update the center-of-mass and total mass of n. 
       Recursively insert the body n in the appropriate quadrant.

    3. If node n is an external node, say containing a body named c, then there are
       two bodies b and c in the same region. Subdivide the region further by creating 
       four children. Then, recursively insert both b and c into the appropriate 
       quadrant(s). Since b and c may still end up in the same quadrant, there may be
       several subdivisions during a single insertion. 
    '''
    def insert(self, b):
        b_idx = self.get_quadrant_index(b)
        if self.num_bodies == 0:
            self.b = b
        elif self.children:
            self.children[b_idx].insert(b)
        else:
            self.subdivide()
            c_idx = self.get_quadrant_index(self.b)
            self.children[c_idx].insert(self.b)
            self.b = None
            self.children[b_idx].insert(b)
        self.num_bodies += 1 
    
    '''
    Finally, update the center-of-mass and total mass of n.
    '''
    def compute_cm(self):
        if self.b:
            self.total_mass = self.b.mass
            self.x_cm = self.b.x
            self.y_cm = self.b.y
        else:
            for ch in self.children:
                ch.compute_cm()
                self.total_mass += ch.total_mass
            for ch in self.children:
                weight = ch.total_mass / self.total_mass
                self.x_cm += ch.x_cm * weight
                self.y_cm += ch.y_cm * weight

In [4]:
bodies = []
size = 100000
G_CONST = 0.005
THOLD = 0.5
        
def create_bodies(num_bodies=5):
    for i in range(num_bodies):
        bodies.append(Body(random.uniform(10,30), \
                           random.uniform(-size, size), \
                           random.uniform(-size, size)))
    return bodies

def build_quad_tree():
    root = QuadTree(-size, size, -size, size)
    for b in bodies: 
        root.insert(b)
    root.compute_cm()
    for b in bodies:
        b.compute_acceleration(root, g=G_CONST, theta=THOLD)
        b.compute_velocity()
    return root

In [5]:
bs = create_bodies()

In [6]:
for b in bs:
    print(b.mass, b.x, b.y)

24.44949046658545 84542.2283176668 -97518.7783658552
15.148713946012514 -37860.57870323955 43331.293706472905
22.97897402822751 -34021.94354899801 -79323.38126842222
17.487871480574277 -92779.90552833455 -40354.94789936682
10.468341729056444 -6535.785784568885 38130.88626872131


In [7]:
rt = build_quad_tree()

In [8]:
def dump(obj):
    for attr in dir(obj):
        print("obj.%s = %r" % (attr, getattr(obj, attr)))

In [9]:
dump(rt)

obj.__class__ = <class '__main__.QuadTree'>
obj.__delattr__ = <method-wrapper '__delattr__' of QuadTree object at 0x7f6f0371cfd0>
obj.__dict__ = {'num_bodies': 5, 'total_mass': 90.53339165045618, 'y_mid': 0.0, 'b': None, 'x_mid': 0.0, 'x_max': 100000, 'x_cm': -10816.51657581098, 'y_cm': -42605.23008092219, 'y_max': 100000, 'children': [<__main__.QuadTree object at 0x7f6f0371cd30>, <__main__.QuadTree object at 0x7f6f0371cc18>, <__main__.QuadTree object at 0x7f6f0371ccc0>, <__main__.QuadTree object at 0x7f6f0371ce10>], 'y_min': -100000, 'size': 200000, 'x_min': -100000}
obj.__dir__ = <built-in method __dir__ of QuadTree object at 0x7f6f0371cfd0>
obj.__doc__ = None
obj.__eq__ = <method-wrapper '__eq__' of QuadTree object at 0x7f6f0371cfd0>
obj.__format__ = <built-in method __format__ of QuadTree object at 0x7f6f0371cfd0>
obj.__ge__ = <method-wrapper '__ge__' of QuadTree object at 0x7f6f0371cfd0>
obj.__getattribute__ = <method-wrapper '__getattribute__' of QuadTree object at 0x7f6f0371cfd0